In [32]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [16]:
#for model registry/logging u need this version of mlfow
import mlflow
print(f"Current MLflow version: {mlflow.__version__}")

Current MLflow version: 2.15.0


In [33]:
#download stopwords and wordnet
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ganesh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ganesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [42]:
#Take out sample data from whole data to perform actions
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
297,"Recently, a friend and I were discussing educa...",positive
467,so yes it is quite nostalgic watching the 1st ...,positive
571,"Yes it's a Fast Times wannabe, but it's still ...",positive
38,I don't understand why this movie was released...,negative
383,1985 was a good year for films - maybe even gr...,positive


In [43]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [44]:
df = normalize_text(df)
df.head()

,review,sentiment
297,recently friend discussing educational ethical...,positive
467,yes quite nostalgic watching st episode one ep...,positive
571,yes fast time wannabe still decent entertainme...,positive
38,understand movie released looked like somethin...,negative
383,good year film maybe even great one missing go...,positive


In [45]:
df['sentiment'].value_counts()

sentiment
negative    256
positive    244
Name: count, dtype: int64

In [46]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [47]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
297,recently friend discussing educational ethical...,1
467,yes quite nostalgic watching st episode one ep...,1
571,yes fast time wannabe still decent entertainme...,1
38,understand movie released looked like somethin...,0
383,good year film maybe even great one missing go...,1


In [48]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [49]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [50]:
X.shape, y.shape

((500, 100), (500,))

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [54]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load from .env file

True

In [55]:
# Read environment variables
MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
DAGSHUB_OWNER = os.getenv("DAGSHUB_OWNER")
DAGSHUB_REPO = os.getenv("DAGSHUB_REPO")

In [56]:
import dagshub

# Set up MLflow and DagsHub
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
dagshub.init(repo_owner=DAGSHUB_OWNER, repo_name=DAGSHUB_REPO, mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-07-14 12:00:19,637 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/ganeshsp7/Capstone-MLOps-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "ganeshsp7/Capstone-MLOps-Project"

2025-07-14 12:00:19,644 - INFO - Initialized MLflow to track repo "ganeshsp7/Capstone-MLOps-Project"


Repository ganeshsp7/Capstone-MLOps-Project initialized!

2025-07-14 12:00:19,648 - INFO - Repository ganeshsp7/Capstone-MLOps-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/f26093aae8c04f5786b1617b07fd47e1', creation_time=1752392356921, experiment_id='0', last_update_time=1752392356921, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [57]:
import mlflow
import logging
import os
import time
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")
        


        # # Set the model name explicitly
        # model_name = "Logistic Regression"

        # mlflow.sklearn.log_model(
        #     sk_model=model,
        #     artifact_path="model",
        #     registered_model_name=model_name
        # )


        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-07-14 12:00:22,192 - INFO - Starting MLflow run...
2025-07-14 12:00:22,607 - INFO - Logging preprocessing parameters...
2025-07-14 12:01:30,910 - INFO - Initializing Logistic Regression model...
2025-07-14 12:01:30,913 - INFO - Fitting the model...
2025-07-14 12:01:30,952 - INFO - Model training complete.
2025-07-14 12:01:30,953 - INFO - Logging model parameters...
2025-07-14 12:01:31,421 - INFO - Making predictions...
2025-07-14 12:01:31,423 - INFO - Calculating evaluation metrics...
2025-07-14 12:01:31,440 - INFO - Logging evaluation metrics...
2025-07-14 12:02:00,503 - INFO - Saving and logging the model...
c:\Users\Ganesh\anaconda3\envs\Mlops1\lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the tradit